# TCPで接続する

In [ ]:
import zmq, msgpack

context = zmq.Context()
socket = context.socket(zmq.REQ)
socket.connect('tcp://localhost:50020')

# Stringをつかって操作する

## 基本的な使い方

In [ ]:
# Recordを開始する
socket.send_string('R')
socket.recv()

In [ ]:
# Recordを停止する
socket.send_string('r')
socket.recv()

In [ ]:
# session名を指定してRecord開始する
socket.send_string('R session_no_namae')
socket.recv()

## そのほかにできること

In [ ]:
#socket.send_string('R')
#socket.send_string('R namae')
#socket.send_string('r')
#socket.send_string('C')
#socket.send_string('c')
#socket.send_string('T `1234.56')
#socket.send_string('t')
socket.send_string('v')
socket.recv()

# より複雑な命令をする

## 基本的な使い方

In [ ]:
# Eye Camera 0を起動させる
n = {'subject': 'eye_process.should_start.0',
     'eye_id' : 0}
socket.send_multipart([('notify.eye_process.should_start.0').encode('utf-8'),
                        msgpack.dumps(n)])
socket.recv()

In [ ]:
# 使いやすいように関数化する
def send_recv_notification(n, socket):
    socket.send_multipart([('notify.%s'%n['subject']).encode('utf-8'),
                           msgpack.dumps(n)])
    return socket.recv()

In [ ]:
# Eye Camera 0を止める
n = {'subject': 'eye_process.should_stop.0',
     'eye_id' : 0}
send_recv_notification(n, socket)

In [ ]:
# EYE CAMERA 1を起動する
n = {'subject': 'eye_process.should_start.1',
     'eye_id' : 1,}
send_recv_notification(n, socket)

In [ ]:
# Recordingを開始する
n = {'subject'     : 'recording.should_start',
     'session_name': 'test_session'}
send_recv_notification(n, socket)
#socket.send_string('R test_session')
#socket.recv()
# と同じ

In [ ]:
# Recordingを停止する
n = {'subject'     : 'recording.should_stop',
     'session_name': 'test_session'}
send_recv_notification(n, socket)
#socket.send_string('r')
#socket.recv()
# と同じ

In [ ]:
# Plugin（Annotation Capture）を起動する
n = {'subject': 'start_plugin',
     'name'   : 'Annotation_Capture'}
send_recv_notification(n, socket)

## Annotationを入れる

In [ ]:
socket.send_string('t')
ts = socket.recv()
n = {'topic'    : 'annotation',
     'label'    : 'raberu_no_namae',
     'timestamp': float(ts)}
socket.send_multipart([(n['topic']).encode('utf-8'),
                        msgpack.dumps(n)])
socket.recv()

In [ ]:
# 関数にまとめる
def send_annotation(n, socket):
    socket.send_string('t')
    ts = socket.recv()
    n['topic'] = 'annotation'
    n['timestamp'] = float(ts)
    socket.send_multipart([(n['topic']).encode('utf-8'),
                            msgpack.dumps(n)])
    socket.recv()

In [ ]:
n = {'label'    : 'session_start',
     'duration' : 1.0,
     'source'   : 'a test script',
     'record'   : True}
send_annotation(n, socket)

# Classにまとめてみた

In [ ]:
import zmq, msgpack
class PupilLabs:    
    def __init__(self, session_name):

        def send_command(n, socket):
            socket.send_multipart([('notify.%s'%n['subject']).encode('utf-8'), msgpack.dumps(n)])
            return socket.recv()
        
        self.send_command = send_command
        self.session_name = session_name
        context = zmq.Context()
        self.socket = context.socket(zmq.REQ)
        self.socket.connect('tcp://localhost:50020')

        # EYE CAMERA 1 (/2) ACTIVATE
        n = {'subject': 'eye_process.should_start.0',
             'eye_id' : 0}
        send_command(n, self.socket)
        
        # EYE CAMERA 2 (/2) INACTIVATE
        n = {'subject': 'eye_process.should_stop.0',
             'eye_id' : 1}
        send_command(n, self.socket)

        # START PLUGIN
        n = {'subject': 'start_plugin',
             'name'   : 'Annotation_Capture'}
        send_command(n, self.socket)

    def init_recording(self):
        n = {'subject'     : 'recording.should_start',
             'session_name': self.session_name}
        self.send_command(n, self.socket)

    def stop_recording(self):
        n = {'subject'     : 'recording.should_stop'}
        self.send_command(n, self.socket)

    def send_annotation(self, label):
        self.socket.send_string('t')
        ts = float(self.socket.recv())
        n = {'topic'    : 'annotation',
             'label'    : label,
             'timestamp': ts}
        self.socket.send_multipart([(n['topic']).encode('utf-8'), msgpack.dumps(n)])
        self.socket.recv()

In [ ]:
from time import sleep
pupillabs = PupilLabs('test')
sleep(5)
pupillabs.init_recording()
for i in range(3):
    sleep(1)
    pupillabs.send_annotation('test {}'.format(i))
sleep(1)
pupillabs.stop_recording()

# pupil labsからLSLでデータを送る

## LSL Pluginを起動する

In [ ]:
import zmq, msgpack
context = zmq.Context()
socket = context.socket(zmq.REQ)
socket.connect('tcp://localhost:50020')

def send_recv_notification(n, socket):
    socket.send_multipart([('notify.%s'%n['subject']).encode('utf-8'),
                           msgpack.dumps(n)])
    return socket.recv()

n = {'subject': 'start_plugin',
     'name'   : 'Pupil_LSL_Relay'}
send_recv_notification(n, socket)

In [ ]:
# オプションをオフにする
n = {'subject': 'start_plugin',
     'name'   : 'Pupil_LSL_Relay',
     'args'   : {'relay_pupil'        : True,
                 'relay_gaze'         : True,
                 'relay_notifications': False}}
send_recv_notification(n, socket)